<h1>Manhole Object Detection Model Test </h1>

<p>This is script will let you test the model on a set of images collected from Google Street View we will explain in each cell the different steps </p>

<h3>Step 1</h3>
<p> First of all, we need to configure the different paths and import scripts.</p>
<h4>1.1 Specify which device we will use </h4>

In [ ]:
! pip install tensorflow

In [1]:
import os # importing OS in order to make GPU visible
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # do not change anything in here
# specify which device you want to work on. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

In [2]:
import tensorflow as tf # import tensorflow

# checking that GPU is found
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


In [3]:
# other import
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import cv2

<h4>1.2 Import libraries provided by Tensorflow API </h4>

<p>we will import scripts that were already provided by Tensorflow API.</p>

In [4]:
import sys # importing sys in order to access scripts located in a different folder

path2scripts = 'Tensorflow/models/research/' # provide pass to the research folder
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

In [5]:
#first you need to install some libraries
! pip install tf-slim
! pip install tf-models-official
! pip install tensorflow-io

In [6]:
# importing all scripts that will be needed to export your model and use it for inference
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

<h4>1.3 Import trained model </h4>

In [7]:
#  specify two pathes: to the pipeline.config file and to the folder with trained model.

path2config ='Tensorflow/workspace/training_demo/exported-models/my_modelfinal/pipeline.config'
path2model = 'Tensorflow/workspace/training_demo/exported-models/my_modelfinal/checkpoint'

In [8]:
configs = config_util.get_configs_from_pipeline_file(path2config) # importing config
model_config = configs['model'] # recreating model config
detection_model = model_builder.build(model_config=model_config, is_training=False) # importing model

In [9]:
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)# getting checkpoints 
ckpt.restore(os.path.join(path2model, 'ckpt-0')).expect_partial()

In [10]:
#path to label map should be provided. Category index will be created based on labal map file
path2label_map = 'Tensorflow/workspace/training_demo/annotations/label_map.pbtxt' # TODO: provide a path to the label map file
category_index = label_map_util.create_category_index_from_labelmap(path2label_map,use_display_name=True)

<h3>Step 2</h3>
<p>In this step we will detect manholes and their coordinates we will save them in csv file</p>
<h4>2.1 Support functions </h4>

In [11]:
#this function will detect the objects of models 
def detect_fn(image):


    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

In [12]:
#this function load image into numpy array 
def load_image_into_numpy_array(path):

    return np.array(Image.open(path))

In [13]:
#this function will detect manholes and save the coordinates of boxes that will be used after for coordinates detection
def save_detect_img(image_path,k,local):

    print('Running inference for {}... '.format(image_path), end='')
   
    image_np = load_image_into_numpy_array(image_path)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}

    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    #visualize the boxes in image of 
    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=0.25,
            agnostic_mode=False,
            line_thickness=5)
    
    # This is the way I'm getting my coordinates
    boxes = detections['detection_boxes']
    # get all boxes from an array
    max_boxes_to_draw = boxes.shape[0]
    # get scores to get a threshold
    scores = detections['detection_scores']
    # this is set as a default but feel free to adjust it to your needs
    min_score_thresh=.2
    # iterate over all objects found
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        # 
        if scores is None or scores[i] > min_score_thresh:
            # boxes[i] is the box which will be drawn
            (ymin,xmin,ymax,xmax)=boxes[i]
            # save coordinates 
            coordonn=(ymin,xmin,ymax,xmax,i,scores[i])
            local.append(coordonn)  
    path_detect='image_detct_part_'+str(k)+'.jpg'
    cv2.imwrite( path_detect,cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    return path_detect
 

In [14]:
#this function will calculate goegraphic coordinate of image using metadata of camera the height and the width of image and center of the box  
import math
def count_geo_coordinate(lanc,latc,hc,yawc,w,h,y,x):
    #calculate z 
    z= -h/math.tan(-y*(math.pi/h)+(math.pi/2))
    #calculate ex
    ex=math.sin(math.sin(x*(math.pi/2)-math.pi+yawc))*z
    #calculate ey
    ey=math.cos(math.cos(x*(math.pi/2)-math.pi+yawc))*z
    #calculate latitude
    lat=latc+math.asin(ey/6371)
    #calculate longitude
    lang=lanc+math.asin(ex/(math.cos(latc))*1/6370)
    
    return lat,lang

In [15]:
#this function will help us to detect cropped boxes and save their coordinates in list (li)
#(ymin,xmin,ymax,xmax,i,score(i))
def detect_cropped_boxes(mat,li):
    
    A =mat[0][1]
    B= mat[1][1] #cropped box will be in two boxes consecutive the cropped boxes will have the same Y and the Xmax of the
    #second will be the Xmin of the second 
    for i in range(len(A)):
        for j in range(len(B)):
            if ( (A[i][0])==(B[j][0]) and (A[i][2])==(B[j][2])):
                if (A[i][3]== B[j][1]):
                    return True 
                    li.append((mat[0][0],mat[1][0],i,j))

    B= mat[1][1]
    C= mat[2][1]
    for l in range(len(B)):
        for k in range(len(C)):
            if ( (B[l][0])==(C[k][0]) and (B[l][2])==(C[k][2])):

                if (B[l][3]== C[k][1]):
                    li.append((mat[1][0],mat[2][0],l,k))
                    return True

In [16]:
#this function will update the list of boxes we will save the box in the image where it will have the highest percentage 
#we will save the index of the redondant box in order to delete it after 
#[image_path][ymin,xmin,ymax,xmax,i,score(i)]==>mat
#(image_1,image_2,box_in_img1,box_in_img2)

def update_mat(mat,table_index,li): 
    for j in [0,1]:
        for i in range(len(li)):
            if (mat[j][0]==li[i][0]):
                K=mat[j][1]
                L=mat[j+1][1]
                kl=list(K[li[i][2]])
                ll=list(L[li[i][3]])
                if (kl[5]>ll[5]):
                    kl[3]=ll[3]
                    K[li[i][2]]=tuple(kl)
                    table_index.append((j+1,li[i][3]))
                else:
                    ll[1]=kl[1]
                    L[li[i][3]]=tuple(ll)
                    table_index.append((j,li[i][2]))
        matl=list(mat[j])
        matl[1]=K
        mat[j]=tuple(matl)
        matl2=list(mat[j+1])
        matl2[1]=L
        mat[j+1]=tuple(matl2)
               

In [17]:
# this function will delete the redondant box 
def delete_elem(mat,table_index):
    
    
    for i in range(len(table_index)):
        matl=list(mat[table_index[i][0]])
        matl[1].pop(table_index[i][1])
        mat[table_index[i][0]]=tuple(matl)


   
    
                

In [18]:
#this function will let us save the coordinates of detected 
import csv
def save_detect(image, latc, lonc,rotation,num_box, percentage,xo,yo,lato,lano):
    with open('manhole.csv', 'a', encoding='UTF8') as f:
        writer = csv.writer(f)
        data=[image, latc, lonc,rotation,num_box, percentage,xo,yo,lato,lano]
        writer.writerow(data)

<h3>Main program</h3>
<p>First, we will divide our image into 3 parts because the model was trained with images that were cropped into 3 parts</p><p>Second, we will use save_detect_img to detect the manholes and save their coordinates and insert them in matrix that represent the cropped image and the different boxes that has  </p>
<p> Once we detect we will verifiy if there were cropped boxes</p>
<p> we will detect cropped boxes using  detect_cropped_boxes </p>
<p> we will update the matrix of the boxes using update_mat  </p>
<p> we will delete redundant boxes after</p>
            
     
       
        
           



In [22]:
image_path='img.jpg'
img= cv2.imread(image_path)
mat=[]
height = img.shape[0]
width = img.shape[1]
#here we cropp the images and detect the manholes 
for i in [0,1,2]:
    #here we divide our img 
    cropped_image = img[int(height/2):height, int(width*i/3):int(width*(1+i)/3)]
    path_cropped=image_path[:len(image_path)-4]+'_cropped_'+str(i)+'.jpg'
    cv2.imwrite(path_cropped, cropped_image)
    local=[]
    path_detect=save_detect_img(path_cropped,i,local)
    mat.append((path_detect,local))
li=[]
table_index=[]
#here we detect cropped boxes and delete redundant ones 
if (detect_cropped_boxes(mat,li)):
    update_mat(mat,table_index,li)
    delete_elem(mat,table_index)
local=[]
#here we calculate geolocalization and save them in file 


for j in range(len(mat)):
    if (len(mat[j][1])!=0):
        local=mat[j][1]
        for loc in local:
            ymin=loc[0]
            xmin=loc[1]
            ymax=loc[2]
            xmax=loc[3]
            i=loc[4]
            
            score=loc[5]
            print(score)
            yc=(ymin+ymax)/2
            xc=(xmin*(j+1)+xmax*(j+1))/2   
            t=count_geo_coordinate(43.499715, 3.719969,2.5,31,5461,4096,yc,xc)
            print(t)
            save_detect(mat[j][0], 43.499715,3.719969,31,i, score,xc,yc,t[0],t[1])
        
        


    

    

Running inference for img_cropped_0.jpg... Running inference for img_cropped_1.jpg... Running inference for img_cropped_2.jpg... 0.68334156
(3.7199057790954293, 43.499649359277605)


PermissionError: [Errno 13] Permission denied: 'manhole.csv'

22
